In [1]:
from util import *
import numpy as np
import time
import csv
import xgboost as xgb
import warnings
from sklearn.model_selection import StratifiedShuffleSplit
warnings.filterwarnings('ignore')
import nltk
from joblib import dump, load
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
import sys, os, re
%matplotlib notebook
from xgboost import plot_importance
import matplotlib.pyplot as plt
lex = liwc.parse_liwc("2015")


trial = ['try', 'tried', 'I\'ve', 'trying', 'therapist', 'med', 'therapy', 'doctor', 'pill', 'doc']
mark = ['!', ':)', '&lt', 'Wow', 'wow']
unknow = ['what to do', 'I don\'t know', 'not sure', 'what else to do', 'what\'s the point']

lonely = ['lonely', 'alone', 'loneliness', 'no friend', 'no one', '0 friend', 'No friend', 
         'nobody', 'no fucking friend', 'dont have friend', 'dont have any friend', 'don\'t have any friend'] 
sadness = ['sad', 'tired', 'horrible', 'terrified', 'scared', 'afraid', 'frustrated', 'awful',
                'pathetic', 'upset', 'fear', 'boring', 'Boring', 'miserable', 'tiring', 'feel down']
feeling = ['feel']
# Satisfaction
thank_words = ['Thank', 'thank', 'it help', 'helpful', 'it does help', 'That helps', 'supportive']
agree_word = ['means a', 'feel you', 'response', 'advice', 'you are right', 'Me too',
              'the same feeling', 'feel similar', 'the same thing', 'exactly', 'Exactly']

def get_vector(doc, cats):
    dic = extract(lex, doc)
    vec = np.zeros(len(cats))
    for i in range(len(cats)):
        if cats[i] in dic.keys():
            vec[i] = dic[cats[i]]
# Feature of sentence count, weithed by 3
    sentences = sent_tokenize(doc)
    number_of_sentences = len(sentences)
    if number_of_sentences > 3:
        vec = np.append(vec, [1])
    else:
        sentence_rate = number_of_sentences / 3
        vec = np.append(vec, [sentence_rate])
            
# Feature of words count, using twitter curpons, weighted by 30
    tknzr = TweetTokenizer()
    words = tknzr.tokenize(doc)
    number_of_words = len(words)
    if number_of_words > 30:
        vec = np.append(vec, [1])
    else:
        word_rate = number_of_words / 30   
        vec = np.append(vec, [word_rate])

# unknow
    num_words = 0
    for i in unknow:
        j = doc.count(i)
        num_words += j
    rate_1 = num_words / number_of_words
    vec = np.append(vec, [rate_1])  

# trial
    num_words = 0
    for i in trial:
        if doc.count(i) > 0:
            num_words = 1
            break
    vec = np.append(vec, [num_words]) 

# mark
    num_words = 0
    for i in mark:
        j = doc.count(i)
        num_words += j
    if num_words > 2:
        rate_1 = 1
    else:
        rate_1 = num_words / 2
    vec = np.append(vec, [rate_1])  

# lonely
    num_words = 0
    for i in lonely:
        if doc.count(i) > 0:
            num_words = 1
            break
    vec = np.append(vec, [num_words]) 

# sadness
    num_words = 0
    for i in sadness:
        if doc.count(i) > 0:
            num_words = 1
            break
    vec = np.append(vec, [num_words]) 

# feeling
    num_words = 0
    for i in feeling:
        j = doc.count(i)
        num_words += j
    if num_words > 2:
        rate_1 = 1
    else: 
        rate_1 = num_words / 2
    vec = np.append(vec, [rate_1])  

# Negation binary
    negation_list = ["not", "no", 'nobody', 'nothing', 'none', 'never', 'neither', 'nor', 'nowhere', 'hardly', 'scarcely', 'barely',  
                     "don\'t", "didn\'t", "isn\'t", 'wasn\'t', 'shouldn\'t', 'wouldn\'t', 'couldn\'t', 'doesn\'t']
    num_words = 0
    for i in negation_list:
        j = re.search(i, doc, re.I)
        num_words += 1
    if num_words > 2:
        rate = 1
    else:
        rate = num_words / 2
    vec = np.append(vec, [rate]) 
    
# thank_words
    num_words = 0
    for i in thank_words:
        if doc.count(i) > 0:
            num_words = 1
            break
    vec = np.append(vec, [num_words])

# agree
    num_words = 0
    for i in agree_word:
        if doc.count(i) > 0:
            num_words = 1
            break
    vec = np.append(vec, [num_words])
        
    return vec


LIWC_FILES = {
  "2007": "LIWC/LIWC2007_English100131.dic",
  "2015": "LIWC/LIWC2015_English.dic",
}
CAT_DELIM = "%"
f = open(LIWC_FILES["2015"])
LIWC_all = [] # This stores all the 73 LIWC features
cats_section = False
for l in f:
    l = l.strip()
    if l == CAT_DELIM: 
        cats_section = not cats_section
        continue
    if cats_section:
        try:
            i, cat = l.split("\t")
            cat = cat.split()[0]
            LIWC_all.append(cat)
        except: pass # likely hierarchical category tags
    else:
        break
black_features = ['male', 'number', 'money', 'shehe', 'reward', 'number', 'assent', 'relig']
liwc_features = []
for i in LIWC_all:
    if i not in black_features:
        liwc_features.append(i)

print("Get prepared")

Get prepared


In [2]:
satisfaction_model = load('models/satisfaction_model.joblib')
# Predicting estimate satisfaction in the support-seekers' reply to their received comment
example_reply = "That makes sense. Thank you for your quick answer :) I don't mind answering questions so I'm a little relieved now. Thanks again!" 
features_satisfaction = get_vector(example_reply, liwc_features)
predict_satisfaction = satisfaction_model.predict(np.array(features_satisfaction.reshape(1, -1)))
#  The regression model are trained on data with labels on 1 - 7 scale, the model may ecceeed this scale a little bit
print('The estimated satisfaction of support-seekers expressed in the example reply is: ')
print(predict_satisfaction)

The estimated satisfaction of support-seekers expressed in the example reply is: 
[6.17460756]
